### Гипотеза

Покупает ли человек по акциям товары, за 1 рубль или по рекомендациям

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:85% !important; }</style>"))
display(HTML("<style>.prompt { min-width:10ex !important; }</style>"))
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.float_format', lambda x: '%.6f' % x) 

/tmp/ipykernel_100981/437858164.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
/tmp/ipykernel_100981/437858164.py:1: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


In [3]:
df_train = pd.read_parquet('../data/train_dataset_hackaton2023_train.gzip')
df_test = pd.read_parquet('../data/test_dataset_hackaton2023_test.gzip')

In [4]:
df_train.shape[0]

12129384

In [7]:
def calculate_promo_features(dataset, ids_cols, df_test, ids_cols_test):
    dataset["1rub"] = dataset["dish_name"].apply(lambda x: "1RUB" in x).astype(int)
    dataset["recommendation"] = dataset["dish_name"].apply(lambda x: "up " in x.lower() or "lim." in x.lower() or "ord." in x.lower()).astype(int)
    dataset['clean_dish'] = dataset["dish_name"].apply(lambda x: x.lower().replace("1rub.", "").replace("up ", "").replace("lim.", "").replace("ord.", ""))
    df_dish_sum = dataset.groupby(["clean_dish"], as_index=True).agg({"revenue": ["median"]})
    df_dish_sum.columns = ["price_median"]
    df_dish_sum.reset_index(inplace=True)
    
    dataset = dataset.merge(df_dish_sum, on='clean_dish', how='left')
    dataset["promo"] = (dataset["revenue"] < dataset["price_median"]).astype(int)
    
    df_sum_agg = dataset.groupby(ids_cols, as_index=True).agg({"promo": ["max"], "1rub": ["max"], "recommendation": ["max"]})
    df_sum_agg.columns = ["was_promo", "was_1rub", "was_rec"]
    df_sum_agg.reset_index(inplace=True)
    
    new_ids_cols = list(set(ids_cols).difference(["startdatetime"]))
    
    df_feat = df_sum_agg.groupby(new_ids_cols, as_index=True).agg({"was_promo": ["mean", "std"], "was_1rub": ["mean", "std"], "was_rec": ["mean", "std"]})
    df_feat.columns = [f"{feat}_{func}" for feat in ["was_promo", "was_1rub", "was_rec"] for func in ["mean", "std"]]
    df_feat.reset_index(inplace=True)
    
    df_test["1rub"] = df_test["dish_name"].apply(lambda x: "1RUB" in x).astype(int)
    df_test["recommendation"] = df_test["dish_name"].apply(lambda x: "up " in x.lower() or "lim." in x.lower() or "ord." in x.lower()).astype(int)
    df_test['clean_dish'] = df_test["dish_name"].apply(lambda x: x.lower().replace("1rub.", "").replace("up ", "").replace("lim.", "").replace("ord.", ""))
    
    df_test = df_test.merge(df_dish_sum, on='clean_dish', how='left')
    df_test["promo"] = (df_test["revenue"] < df_test["price_median"]).astype(int)
    
    df_sum_agg_test = df_test.groupby(ids_cols_test, as_index=True).agg({"promo": ["max"], "1rub": ["max"], "recommendation": ["max"]})
    df_sum_agg_test.columns = ["was_promo", "was_1rub", "was_rec"]
    df_sum_agg_test.reset_index(inplace=True)
    
    new_ids_cols = list(set(ids_cols_test).difference(["startdatetime"]))
    
    df_feat_test = df_sum_agg_test.groupby(new_ids_cols, as_index=True).agg({"was_promo": ["mean", "std"], "was_1rub": ["mean", "std"], "was_rec": ["mean", "std"]})
    df_feat_test.columns = [f"{feat}_{func}" for feat in ["was_promo", "was_1rub", "was_rec"] for func in ["mean", "std"]]
    df_feat_test.reset_index(inplace=True)
    
    return df_feat, df_feat_test

In [8]:
df_train_feat, df_test_feat = calculate_promo_features(df_train, ["customer_id", "buy_post", "startdatetime"], df_test, ["customer_id", "startdatetime"])

In [9]:
df_train_feat.describe()

,buy_post,customer_id,was_promo_mean,was_promo_std,was_1rub_mean,was_1rub_std,was_rec_mean,was_rec_std
count,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,0.718190,20246346.095608,0.637119,0.326594,0.048040,0.092784,0.277783,0.282402
std,0.449882,11496610.350691,0.318553,0.250455,0.109195,0.187361,0.298250,0.257271
min,0.000000,29891.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,10731681.250000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,21725643.000000,0.666667,0.447214,0.000000,0.000000,0.222222,0.404520
75%,1.000000,28452387.000000,1.000000,0.547723,0.000000,0.000000,0.500000,0.534522
max,1.000000,46661804.000000,1.000000,0.707107,1.000000,0.707107,1.000000,0.707107


In [10]:
df_test_feat.describe()

,customer_id,was_promo_mean,was_promo_std,was_1rub_mean,was_1rub_std,was_rec_mean,was_rec_std
count,112334.000000,112334.000000,112334.000000,112334.000000,112334.000000,112334.000000,112334.000000
mean,20748611.321951,0.627664,0.326477,0.046562,0.088274,0.273133,0.278968
std,11582578.505840,0.323061,0.254960,0.109459,0.186229,0.297515,0.261279
min,52341.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11041231.250000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
50%,22155898.000000,0.666667,0.462910,0.000000,0.000000,0.222222,0.380693
75%,28861910.500000,1.000000,0.577350,0.000000,0.000000,0.500000,0.547723
max,46668221.000000,1.000000,0.707107,1.000000,0.577350,1.000000,0.707107


### Information value

IV < 0,02 — отсутствует;

0,02 ≤ IV <0,1 — низкая;

0,1 ≤ IV < 0,3 — средняя;

IV ≥ 0,3 — высокая.

In [11]:
def calculate_information_value(dataset, feats, target_col):
    iv_by_feat = {}
    temp = dataset.copy()
    for feat in feats:
        temp["quant_range"] = pd.qcut(
            x=temp[feat], q=[0, 0.25, 0.5, 0.75, 1.0], duplicates="drop"
        )

        bins = {}
        for i, bin in enumerate(temp["quant_range"].unique()):
            bins[bin] = i

        temp["bin"] = temp["quant_range"].apply(lambda x: bins[x])

        iv = (
            pd.crosstab(temp["bin"], temp[target_col], normalize="columns")
            .assign(woe=lambda dfx: np.log(dfx[1] / dfx[0]))
            .assign(iv=lambda dfx: np.sum(dfx["woe"] * (dfx[1] - dfx[0])))
        )["iv"].unique()[0]

        iv_by_feat[feat] = [iv]
    df = pd.DataFrame(iv_by_feat).T
    df.reset_index(inplace=True)
    df.columns = ["feature", "IV"]
    return df

In [13]:
feats = list(set(df_train_feat.columns).difference(["customer_id", "buy_post"]))

In [14]:
iv_df = calculate_information_value(df_train_feat, feats, "buy_post")

In [15]:
iv_df.sort_values(by="IV", ascending=False)

,feature,IV
3,was_promo_std,0.120205
5,was_rec_std,0.090511
0,was_promo_mean,0.034826
2,was_rec_mean,0.003347
1,was_1rub_std,0.000000
4,was_1rub_mean,0.000000
